# Well matcher

helps to identify correct wells between four considerably large datasets. Results in dataset of oil&gas wells matched by various criteria and conditions order.
Provides a little statistics in the end showing which dataset was the most relevant.
Uses a fuzzywuzzy library which features words and character arrays percetnage matching.
Saves a result as an excel file in given location.
Came out to be a real help in one of company's internal data migration processes.

In [4]:
import pandas as pd
import numpy as np
from tkinter import Tk 
from tkinter.filedialog import askopenfilename, askdirectory
import os
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Tk().withdraw() 
filename_mxbg_base = askopenfilename(title='SELECT FILE FOR MXBG DATA BASE')# show an "Open" dialog box and return the path to the selected file
filename_siepDZ_base = askopenfilename(title='SELECT FILE FOR SIEP DZ DATA BASE')
filename_siepMASTER_base = askopenfilename(title='SELECT FILE FOR SIEP MASTER DATA BASE') # show an "Open" dialog box and return the path to the selected file
filename_ihs_base = askopenfilename(title='SELECT FILE FOR IHS DATA BASE')


mxbg = pd.read_csv(filename_mxbg_base)
siepDZ = pd.read_csv(filename_siepDZ_base)
siepMASTER = pd.read_csv(filename_siepMASTER_base,sep=";")
ihs = pd.read_csv(filename_ihs_base,sep=';',encoding="ISO-8859-1")


lista1=[]
DZWellName_choices=[]
DZWellCommonName_choices=[]
MASTERWellName_choices=[]
MASTERWellCommonName_choices=[]
IHSWellName_choices=[]
IHSAlternateWellName_choices=[]


for row in ihs['Well Name']:
    row=row.upper()
    lista1.append(row)

for i in siepDZ['WELL NAME']:
    DZWellName_choices.append(i)
    
for i in siepDZ['WELL COMMON NAME']:
    DZWellCommonName_choices.append(i)
    
for i in siepMASTER['WELL NAME']:
    MASTERWellName_choices.append(i)
    
for i in siepMASTER['WELL COMMON NAME']:
    MASTERWellCommonName_choices.append(i)
    
for i in ihs['Well Name']:
    IHSWellName_choices.append(i)

for i in ihs['Alternate Well Name']:
    IHSAlternateWellName_choices.append(i)

ihs.loc[:,'Well Name'] = pd.Series(lista1, index=ihs.index)

In [5]:
source_project=[]
source_field=[]
source_well =[]
UWBI_mxbg=[]
target_project =[]
target_field=[]
target_well =[]
found=[]


for idx, row in mxbg.iterrows(): 
    source_well.append(row['WELL NAME'])
    source_field.append(row['FIELD NAME'])
    source_project.append(row['RECALL PROJECT NAME'])
    
    x_DZ_WN=process.extractOne(row['WELL NAME'], DZWellName_choices)
    DZsameUWBI = siepDZ.loc[siepDZ['WELL UWBI'] == row['WELL UWBI']]
    x_DZ_WCN=process.extractOne(row['WELL NAME'], DZWellCommonName_choices)
    x_MASTER_WN=process.extractOne(row['WELL NAME'], MASTERWellName_choices)
    MASTERsameUWBI = siepMASTER.loc[siepMASTER['WELL UWBI'] == row['WELL UWBI']]
    x_MASTER_WCN=process.extractOne(row['WELL NAME'], MASTERWellCommonName_choices)
    x_IHS_WN=process.extractOne(row['WELL NAME'], IHSWellName_choices)
    x_IHS_AWN=process.extractOne(row['WELL NAME'], IHSAlternateWellName_choices)
                                
    
    indexSIEP = (DZWellCommonName_choices.index(x_DZ_WCN[0]))
    indexMASTER=(MASTERWellCommonName_choices.index(x_MASTER_WCN[0]))
    indexIHS=(IHSAlternateWellName_choices.index(x_IHS_AWN[0]))
    
    if x_DZ_WN[1]<95:
        
        
        if DZsameUWBI.empty:
            
            if x_DZ_WCN[1]<95:
                
                
                if x_MASTER_WN[1]<95:
                    
                    
                    if MASTERsameUWBI.empty:
                       
                        if x_MASTER_WCN[1]<95:
                            UWBI_mxbg.append('')
                            
                            if x_IHS_WN[1]<95:
                                
                                if x_IHS_AWN[1]<95:
                                    target_well.append('not found')
                                    found.append('not found')
                                    target_field.append('not found')
                                    target_project.append('not found')
                                
                                else:
                                    target_well.append(IHSWellName_choices[indexIHS])
                                    found.append('IHS (Alternate Well Name)')
                                    target_field.append('IHS')
                                    target_project.append('IHS')
                                
                            else:
                                target_well.append(x_IHS_WN[0])
                                found.append('IHS (Well Name)')
                                target_field.append('IHS')
                                target_project.append('IHS')
                                
                                
                                
                        else:
                            target_well.append(MASTERWellName_choices[indexMASTER])               
                            UWBI_mxbg.append(row['WELL UWBI'])                                                                
                            found.append('SIEP MASTER (Common Name)')
                            target_field.append(siepMASTER.iloc[0]['FIELD NAME'])                
                            target_project.append(siepMASTER.iloc[0]['RECALL PROJECT NAME'])
                            
                    else:
                        target_well.append(MASTERsameUWBI.iloc[0]['WELL NAME'])    
                        UWBI_mxbg.append(row['WELL UWBI'])                                                        
                        found.append('SIEP MASTER (UWBI)')
                        target_field.append(siepMASTER.iloc[0]['FIELD NAME'])                
                        target_project.append(siepMASTER.iloc[0]['RECALL PROJECT NAME'])
                
                    
                else:
                    target_well.append(x_MASTER_WN[0])
                    UWBI_mxbg.append(row['WELL UWBI'])
                    found.append('SIEP MASTER (Well Name)')
                    target_field.append(siepMASTER.iloc[0]['FIELD NAME'])                
                    target_project.append(siepMASTER.iloc[0]['RECALL PROJECT NAME'])
                    
            else:
                target_well.append(DZWellName_choices[indexSIEP])               
                UWBI_mxbg.append(row['WELL UWBI'])                                                                
                found.append('OFDB (Common Name)')
                target_field.append(siepDZ.iloc[0]['FIELD NAME'])                
                target_project.append(siepDZ.iloc[0]['RECALL PROJECT NAME'])
            
        else:
            target_well.append(DZsameUWBI.iloc[0]['WELL NAME'])    
            UWBI_mxbg.append(row['WELL UWBI'])                                                        
            found.append('OFDB (UWBI)')
            target_field.append(siepDZ.iloc[0]['FIELD NAME'])                
            target_project.append(siepDZ.iloc[0]['RECALL PROJECT NAME'])
            
    else:
        target_well.append(x_DZ_WN[0])
        UWBI_mxbg.append(row['WELL UWBI'])
        found.append('OFDB (Well Name)')
        target_field.append(siepDZ.iloc[0]['FIELD NAME'])                
        target_project.append(siepDZ.iloc[0]['RECALL PROJECT NAME'])
        

pd.options.display.float_format = '{:.0f}'.format       
search_result=pd.DataFrame({'SOURCE PROJECT':source_project,'SOURCE FIELD':source_field,'SOURCE WELL':source_well,'UWBI':UWBI_mxbg,'TARGET PROJ':target_project,'TARGET FIELD':target_field,'TARGET WELL':target_well,'FOUND ON':found})

search_result
     
        


,SOURCE PROJECT,SOURCE FIELD,SOURCE WELL,UWBI,TARGET PROJ,TARGET FIELD,TARGET WELL,FOUND ON
0,MXBG_ALGERIA_OFDB,ALGERIA,AIN EL ZANA 1,NaN,SIEP_WELL_MASTER,ALGERIA,AIN EL ZANA-1,SIEP MASTER (Well Name)
1,MXBG_ALGERIA_OFDB,ALGERIA,AIN GOUFAFA 3,NaN,SIEP_WELL_MASTER,ALGERIA,AIN GOUFAFA 3,SIEP MASTER (Well Name)
2,MXBG_ALGERIA_OFDB,ALGERIA,AIN GOUFAFA 4,NaN,SIEP_WELL_MASTER,ALGERIA,AIN GOUFAFA 4,SIEP MASTER (Well Name)
3,MXBG_ALGERIA_OFDB,ALGERIA,ALLENDA NORD 1,NaN,CLDB_DZ_SIEP,ALGERIA,ALLENDA NORD 1,OFDB (Well Name)
4,MXBG_ALGERIA_OFDB,ALGERIA,ALLENDA NORD 2,NaN,SIEP_WELL_MASTER,ALGERIA,ALLENDA NORD 2,SIEP MASTER (Well Name)
5,MXBG_ALGERIA_OFDB,ALGERIA,ASKARENE 101,NaN,SIEP_WELL_MASTER,ALGERIA,ASKARENE-101,SIEP MASTER (Well Name)
6,MXBG_ALGERIA_OFDB,ALGERIA,ASKARENE-102,NaN,SIEP_WELL_MASTER,ALGERIA,ASKARENE-102,SIEP MASTER (Well Name)
7,MXBG_ALGERIA_OFDB,ALGERIA,AZEL 1,NaN,CLDB_DZ_SIEP,ALGERIA,AZEL 1,OFDB (Well Name)
8,MXBG_ALGERIA_OFDB,ALGERIA,AZEL NORD 1,NaN,CLDB_DZ_SIEP,ALGERIA,AZEL NORD 1,OFDB (Well Name)
9,MXBG_ALGERIA_OFDB,ALGERIA,AZEL NORD 2,NaN,CLDB_DZ_SIEP,ALGERIA,AZEL NORD 2,OFDB (Well Name)


In [7]:
search_result.groupby('FOUND ON').size()

FOUND ON
IHS (Alternate Well Name)     4
IHS (Well Name)               3
OFDB (Common Name)           16
OFDB (Well Name)             46
SIEP MASTER (Common Name)    19
SIEP MASTER (Well Name)      24
not found                    14
dtype: int64

In [22]:
with pd.ExcelWriter('Well_Matcher_Alfa-OFDB.xlsx') as writer:                               
    search_result.to_excel(writer,sheet_name='sheet_name1')